In [ ]:
# https://dzone.com/articles/simple-sentiment-analysis-with-nlp

In [35]:
import nltk
import import_data as id
import prepare_data as pre_data
import feature_engineering as fe
import modelling as mod


In [118]:
if __name__ == "__main__":

    print ("\n\n=================== Sentiment Analysis on Movie Reviews ===================")
    print ("\n\n------------------ Data Loading and Preprocessing ------------------")
    # Data Import/Data Load - Import text files
    # Load the Training Data
    path = './data_movie_reviews/train/pos/'
    train_positiveFiles, train_pos_merge_content = id.import_files(path)
    print ("Total of {} Files is loaded with Positive Sentiment for Training".format(len(train_positiveFiles)))

    path = './data_movie_reviews/train/neg/'
    train_negativeFiles, train_neg_merge_content = id.import_files(path)
    print("Total of {} Files is loaded with Negative Sentiment for Training".format(len(train_negativeFiles)))

    # Load the Testing Data
    path = './data_movie_reviews/test/pos/'
    test_positiveFiles, test_pos_merge_content = id.import_files(path)
    print("Total of {} Files is loaded with Positive Sentiment for Testing".format(len(test_positiveFiles)))

    path = './data_movie_reviews/test/neg/'
    test_negativeFiles, test_neg_merge_content = id.import_files(path)
    print("Total of {} Files is loaded with Negative Sentiment for Testing".format(len(test_negativeFiles)))

    print ("\n\n---------------------Dataframe of all the training and testing reviews---------------------\n ")
    reviews = id.create_dataframe(train_pos_merge_content,train_neg_merge_content,
                            test_pos_merge_content,test_neg_merge_content,
                            train_positiveFiles,train_negativeFiles,
                            test_positiveFiles,test_negativeFiles)
    
    print("\n Data Loading and Preprocessing Completed...")






=================== Sentiment Analysis on Movie Reviews ===================


------------------ Data Loading and Preprocessing ------------------
Total of 12500 Files is loaded with Positive Sentiment for Training
Total of 12500 Files is loaded with Negative Sentiment for Training
Total of 12500 Files is loaded with Positive Sentiment for Testing
Total of 12500 Files is loaded with Negative Sentiment for Testing


---------------------Dataframe of all the training and testing reviews---------------------
 
              file  label                                             review
26247  11122_8.txt      1  Fame is one of the best movies I've seen about...
35067  7811_10.txt      1  This movie fully deserves to be one of the top...
34590  7382_10.txt      1  in a time of predictable movies, in which abou...
16668   2501_1.txt      0  I saw this on TV the other nightÂ or rather I...
12196   9728_7.txt      1  I am a huge fan of Simon Pegg and have watched...

 Data Loading and Prep

In [119]:
print (reviews.head())
number_of_rows = 1000
reviews = reviews.iloc[ 0: number_of_rows, : ]

              file  label                                             review
26247  11122_8.txt      1  Fame is one of the best movies I've seen about...
35067  7811_10.txt      1  This movie fully deserves to be one of the top...
34590  7382_10.txt      1  in a time of predictable movies, in which abou...
16668   2501_1.txt      0  I saw this on TV the other nightÂ or rather I...
12196   9728_7.txt      1  I am a huge fan of Simon Pegg and have watched...


## Make Features

- “ “.join() restores the gap between the words in a line and is applied to each split word once the punctuation marks have been stripped off it by the function translate(str.maketrans('', '', string.punctuation))
 
- All words of length < 4 as well as the stopwords are removed from the above processed text 

In [120]:
import string
message_text = ' '.join(list(reviews["review"].values))
message_text = message_text.lower().translate(str.maketrans('', '', string.punctuation))
all_words=message_text.split()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in all_words if not w in stop_words ]

In [121]:
filtered_sentence = [w for w in filtered_sentence if len(w)>=4]
print (filtered_sentence[10])
print (len(filtered_sentence))

design
109833


In [122]:
words_labels = nltk.pos_tag(filtered_sentence)
import pandas as pd
df = pd.DataFrame(words_labels)

In [123]:
nouns_to_be_removed = df.loc[df[1]=='NNP']
print ("No of nouns that are removed", len(nouns_to_be_removed))
print (nouns_to_be_removed)

No of nouns that are removed 48
                    0    1
179              khan  NNP
196              khan  NNP
5113           khanna  NNP
7346          kitchen  NNP
8611             knew  NNP
9785             knox  NNP
14769        kabinett  NNP
14954           amber  NNP
16714            kick  NNP
20892            know  NNP
26340      misfortune  NNP
27584         mockney  NNP
29609          zephyr  NNP
32544           â¨big  NNP
34824           keith  NNP
41188            yarn  NNP
43455            âin  NNP
44068         mamasan  NNP
44152          kattan  NNP
51998           zenon  NNP
56296            kris  NNP
56297   kristofferson  NNP
56371   kristofferson  NNP
58347           maddy  NNP
59552          killer  NNP
61126           zucco  NNP
62315           orton  NNP
63704           queue  NNP
73166          zombie  NNP
76397          valley  NNP
79206            zita  NNP
83346          molina  NNP
83904       gabrielle  NNP
86986          killer  NNP
87801            kono  

## from filtered_sentence remove the nouns.

In [124]:
# the indexes to be removed
nouns_to_be_removed.index

Int64Index([   179,    196,   5113,   7346,   8611,   9785,  14769,  14954,
             16714,  20892,  26340,  27584,  29609,  32544,  34824,  41188,
             43455,  44068,  44152,  51998,  56296,  56297,  56371,  58347,
             59552,  61126,  62315,  63704,  73166,  76397,  79206,  83346,
             83904,  86986,  87801,  90519,  92008,  95627,  96988,  97235,
             99214,  99442, 103833, 103891, 104956, 105904, 105914, 107703],
           dtype='int64')

In [125]:
df = pd.DataFrame(filtered_sentence).drop(nouns_to_be_removed.index)

In [126]:
# df

In [127]:
df.loc[:,0].values

array(['fame', 'best', 'movies', ..., 'heartfilled', 'review',
       'mattercheers'], dtype=object)

In [128]:
# nltk.FreqDist(df.loc[:,0].values)

In [129]:
## frequency counts
# all_words = nltk.FreqDist(filtered_sentence)  # frequency distribution
all_words = nltk.FreqDist(df.loc[:,0].values)
word_features = list(all_words.keys())[:4000]  # take 3000 words as featutes


In [130]:
# word_features

In [131]:
reviews.shape

(1000, 3)

In [132]:
def split_reviews_to_words(rev):
    list_of_docs_in_words = []
    for review in rev:
        words = review.split()
        filtered = [w for w in words if not w in stop_words and len(w) >=4]
        list_of_docs_in_words.append(filtered)
    return list_of_docs_in_words
    
res = split_reviews_to_words(list(reviews['review'].values))
reviews['list_of_words'] = res
documents = reviews.loc[:, ['list_of_words', 'label']]
documents.head(2)

,list_of_words,label
26247,"[Fame, best, movies, I've, seen, Performing, A...",1
35067,"[This, movie, fully, deserves, Hindi, comedies...",1


In [133]:
# for each word in the document assign true/false.
def find_features(document):
    words = set(document)   # get list of unique words
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features
#(words :(true if in  3000 features)  and category)
featuresets = [(find_features(rev), category) for rev, category in documents.values]

In [134]:
training_range = int (number_of_rows/1.3)

training_set = featuresets[:training_range]
testing_set = featuresets[training_range:]

In [135]:
# training_set[3]

In [136]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 77.48917748917748


In [137]:
classifier.show_most_informative_features(30)

Most Informative Features
                   waste = True                0 : 1      =     11.4 : 1.0
                   worst = True                0 : 1      =     11.1 : 1.0
                    lame = True                0 : 1      =      9.8 : 1.0
                straight = True                0 : 1      =      9.8 : 1.0
                  wasted = True                0 : 1      =      8.4 : 1.0
                   spent = True                0 : 1      =      7.8 : 1.0
                 deserve = True                0 : 1      =      7.1 : 1.0
                  poorly = True                0 : 1      =      7.1 : 1.0
               redeeming = True                0 : 1      =      7.1 : 1.0
                   sweet = True                1 : 0      =      6.9 : 1.0
                hospital = True                1 : 0      =      6.9 : 1.0
                    sent = True                1 : 0      =      6.9 : 1.0
                  nudity = True                0 : 1      =      6.4 : 1.0

In [138]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

####  this classifier is suitable for discrete data. The difference is that while MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolean features.  here we have positive/negative as boolean features
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

MNB_classifier accuracy percent: 77.9220779221
BernoulliNB_classifier accuracy percent: 78.7878787879


In [139]:
print("Original Naive Bayes Algorithm accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(30)

Original Naive Bayes Algorithm accuracy percent: 77.48917748917748
Most Informative Features
                   waste = True                0 : 1      =     11.4 : 1.0
                   worst = True                0 : 1      =     11.1 : 1.0
                    lame = True                0 : 1      =      9.8 : 1.0
                straight = True                0 : 1      =      9.8 : 1.0
                  wasted = True                0 : 1      =      8.4 : 1.0
                   spent = True                0 : 1      =      7.8 : 1.0
                 deserve = True                0 : 1      =      7.1 : 1.0
                  poorly = True                0 : 1      =      7.1 : 1.0
               redeeming = True                0 : 1      =      7.1 : 1.0
                   sweet = True                1 : 0      =      6.9 : 1.0
                hospital = True                1 : 0      =      6.9 : 1.0
                    sent = True                1 : 0      =      6.9 : 1.0
       

In [140]:
from sklearn.linear_model import LogisticRegression
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)


LogisticRegression_classifier accuracy percent: 77.0562770563


In [141]:
from sklearn.svm import SVC, LinearSVC
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)


SVC_classifier accuracy percent: 50.2164502165


In [142]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)


LinearSVC_classifier accuracy percent: 71.8614718615
